In [47]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from urllib import request as rq
import urllib
import re
import math
import os
import csv
import time
import json
import pickle
import html5lib
import sys
from sklearn.cross_decomposition import PLSRegression
import matplotlib.pyplot as plt
cwd = os.getcwd()
print(cwd)


/home/liam/Documents/info370


In [50]:
f = open("moviebudget2.html", "r") 
df = pd.read_html(f, index_col=0,header=0, thousands="")
csv = pd.read_csv(open("AA_bestpictures_Liam_Kimiko.csv",'r'),delimiter=";")
df = df[0]

nosign = lambda x: ''.join(x[1:].split(","))

df['Production Budget'] = pd.to_numeric(df['Production Budget'].apply(nosign))
df['Domestic Gross'] = pd.to_numeric(df['Domestic Gross'].apply(nosign))
df['Worldwide Gross'] = pd.to_numeric(df['Worldwide Gross'].apply(nosign))

# removes the spaces at the front of each cell
csv["Movie Name"] = csv["Movie Name"].apply(nosign)
csv["Academy Award"] = csv["Academy Award"].apply(nosign)
csv["Winner"] = csv["Winner"].apply(nosign)
csv["Producer"] = csv["Producer"].apply(nosign)

In [51]:
csv = csv.merge(df,how='left',left_on='Movie Name', right_on='Movie')

In [52]:
print(list(csv.columns.values))
caches = []

['year', 'Academy Award', 'Winner', 'Movie Name', 'Producer', 'IMDB Rating', 'IMDB Votes', 'Tomatometer', 'Average Critic Score', 'Percent Audience Liked', 'Average Audience Score', 'Release Date', 'Movie', 'Production Budget', 'Domestic Gross', 'Worldwide Gross']


In [55]:
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,} 
missing2 = []
gsearch = "http://www.google.com/search?q=site:www.imdb.com+"
csv = csv.sort_values('year',ascending=False)
for i, row in enumerate(csv.values):
    print(row[13])
    break
    if math.isnan(row[13]) or math.isnan(row[15]):
#         print(csv.iloc[i])
        print(row)
        e = False
        mname = str(row[3]).replace("é","e")
        myear = str(row[0])
        url = gsearch+mname.replace(" ","+") + "+"+ myear
        print(url)
        request=urllib.request.Request(url,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read() # The data u need
        soup2 = bs(data, 'html.parser')
#         print(data)
#         print(soup2)
        firstres = soup2.find("div", id="search").find_all("div", class_="g")[0]
#         print(firstres)
#         break
        imdblink = firstres.find("cite").contents[0]
        parts = imdblink.split("/")
        if parts[0] == "www.imdb.com" and parts[1] =='title':
            imdblink = "/".join(parts[:3])
            print(imdblink)
            
            cache = None
            links = firstres.find_all("a")
            for link in links:
                if link.contents[0] == "Cached":
                    cache = link['href']
#                     print(cache)
                    cache = "https://www.google.com"+cache
#                     print(cache)
            if cache == None:
                missing2.append([mname, myear,"No Cache"])
                print("NO CACHE")
            else:
                time.sleep(2)
                request2=urllib.request.Request(cache,None,headers) #The assembled request
                response2 = urllib.request.urlopen(request2)
                datacache = response2.read() # The data u need
                soupcache = bs(datacache, 'html.parser')
                details = soupcache.find("div", {"id":"titleDetails"})
                if not details:
                    missing2.append([mname, myear,"No Details"])
                    print("NO DETAILS")

#                 print(details)
                details = details.get_text()
#                 print(details)
                budget = row[-3]
                gross = row[-1]
                for item in details.split("\n"):
                    if "Budget" in item:
                        try:
                            budget = item.split("$")[1].strip().replace(",", "")
                        except IndexError: 
                            budget = item.split("£")[1].strip().replace(",", "")
                    if "Gross" in item:
                        try:
                            gross = item.split("$")[1].strip().replace(",", "")
                        except IndexError: 
                            gross = item.split("£")[1].strip().replace(",", "")
                print(budget)
                print(gross)
                if budget or gross:
                    row[15] = float(gross)
                    row[13] = float(budget)
                    print(row)
                    csv.iloc[i] = row
                    print(csv.iloc[i])
                else:
                    print("NO Box Office Info")
            
        else:
            print("Wasn't first link!")
            missing2.append([mname, myear,"Wasn't first link!"])
        time.sleep(2)
#     break
csv['Return on Investment'] = csv['Worldwide Gross'].div(csv['Production Budget'])
w = open('AA_4.csv', 'w')
csv.to_csv(w, index=False)
w.close()

3300000.0


In [45]:
csv['Return on Investment'] = csv['Worldwide Gross'].div(csv['Production Budget'])

In [46]:
w = open('AA_3.csv', 'w')
csv.to_csv(w, index=False)
w.close()